# Cross Language Information Retrieval

#### Overview

The aim of this project is to build a cross language information retrieval system (CLIR) which, given a query in German, will be capable of searching text documents written in English and displaying the results in German.

We're going to use machine translation, information retrieval using a vector space model, and then assess the performance of the system using IR evaluation techniques.

Parts of the project are explained as we progress.

#### Data Used

- bitext.(en,de): A sentence aligned, parallel German-English corpus, sourced from the Europarl corpus (which is a collection of debates held in the EU parliament over a number of years). We'll use this to develop word-alignment tools, and build a translation probability table. 

- newstest.(en,de): A separate, smaller parallel corpus for evaulation of the translation system.

- devel.(docs,queries,qrel): A set of documents in English (sourced from Wikipedia), queries in German, and relevance judgement scores for each query-document pair. 

The files are available to check out in the data/clir directory of the repo. 

### Housekeeping: File encodings and tokenisation

Since the data files we use is utf-8 encoded text, we need to convert the strings into ASCII by escaping the special symbols. We also import some libraries in this step as well.

In [2]:
from nltk.tokenize import word_tokenize
from __future__ import division #To properly handle floating point divisions.
import math

def tokenize(line, tokenizer=word_tokenize):
    utf_line = line.decode('utf-8').lower()
    return [token.encode('ascii', 'backslashreplace') for token in tokenizer(utf_line)]

Now we can test out our tokenize function. Notice how it converts the word Über.

In [4]:
tokenize("Seit damals ist er auf über 10.000 Punkte gestiegen.")

['seit',
 'damals',
 'ist',
 'er',
 'auf',
 '\\xfcber',
 '10.000',
 'punkte',
 'gestiegen',
 '.']

With that out of the way, lets get to the meat of the project. 

As mentioned earlier, we're going to build a CLIR engine consisting of information retrieval and translation components, and then evaluate its accuracy.

The CLIR system will:
- **translate queries** from German into English (because our searcheable corpus is in English), using word-based translation, a rather simplistic approach as opposed to the sophistication you might see in, say, *Google Translate*.
- **search over the document corpus** using the Okapi BM25 IR ranking model, a variation of the traditional TF-IDF model.
- **evaluate the quality** of ranked retrieval results using the query relevance judgements.

### Information Retrieval using [Okapi BM25](https://en.wikipedia.org/wiki/Okapi_BM25)

We'll start by building an IR system, and give it a test run with some English queries. 

Here's an overview of the tasks involved:
- Loading the data files, and tokenizing the input.
- Preprocessing the lexicon by stemming, removing stopwords.
- Calculating the TF/IDF representation for all documents in our wikipedia corpus.
- Storing an inverted index to efficiently documents, given a query term.
- Implementing querying with BM25.
- Test runs.

So for our first task, we'll load the devel.docs file, extract and tokenize the terms, and store them in a python dictionary with the document ids as keys. 

In [9]:
import nltk
import re

stopwords = set(nltk.corpus.stopwords.words('english')) #converting stopwords to a set for faster processing in the future.
stemmer = nltk.stem.PorterStemmer() 

#Function to extract and tokenize terms from a document
def extract_and_tokenize_terms(doc):
    terms = []
    for token in tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are faster over sets than lists
            if not re.search(r'\d',token) and not re.search(r'[^A-Za-z-]',token): #Removing numbers and punctuations 
                #(excluding hyphenated words)
                terms.append(stemmer.stem(token.lower()))
    return terms

documents = {} #Dictionary to store documents with ids as keys.

In [10]:
#Reading each line in the file and storing it documents dictionary
f = open('data/clir/devel.docs')
for line in f:
    doc = line.split("\t")
    terms = extract_and_tokenize_terms(doc[1])
    documents[doc[0]] = terms

f.close()

To check if everything is working till now, let's access a document from the dictionary, with the id '290'. 

In [11]:
documents['290']

[u'name',
 u'plural',
 u'ae',
 u'first',
 u'letter',
 u'vowel',
 u'iso',
 u'basic',
 u'latin',
 u'alphabet',
 u'similar',
 u'ancient',
 u'greek',
 u'letter',
 u'alpha',
 u'deriv',
 u'upper',
 u'case',
 u'version',
 u'consist',
 u'two',
 u'less',
 u'vertic',
 u'line',
 u'join',
 u'top',
 u'cross',
 u'middl',
 u'horizont',
 u'bar',
 u'originsth',
 u'earliest',
 u'certain',
 u'ancestor',
 u'aleph',
 u'also',
 u'call',
 u'aleph',
 u'first',
 u'letter',
 u'phoenician',
 u'alphabet',
 u'consist',
 u'entir',
 u'conson',
 u'abjad',
 u'rather',
 u'true',
 u'alphabet',
 u'turn',
 u'origin',
 u'aleph',
 u'may',
 u'pictogram',
 u'ox',
 u'head',
 u'proto',
 u'sinait',
 u'script',
 u'influenc',
 u'egyptian',
 u'hieroglyph',
 u'style',
 u'triangular',
 u'head',
 u'two',
 u'horn',
 u'extend',
 u'egyptian',
 u'cretan',
 u'phoenician',
 u'aleph',
 u'semit',
 u'greek',
 u'alpha',
 u'etruscan',
 u'roman',
 u'cyril',
 u'boeotian',
 u'--',
 u'bc',
 u'greek',
 u'uncial',
 u'latin',
 u'ad',
 u'uncial',
 u'pho

Now we'll build an inverted index for the documents, so that we can quickly access documents for the terms we need. 

In [12]:
#Building an inverted index for the documents

from collections import defaultdict
    
inverted_index = defaultdict(set)

for docid, terms in documents.items():
    for term in terms:
        inverted_index[term].add(docid)    

To test it out, the list of documents containing the word 'ship':

In [13]:
inverted_index['ship']

{'100011',
 '10046',
 '10049',
 '100782',
 '101347',
 '101906',
 '102231',
 '10482',
 '106130',
 '106574',
 '106675',
 '106806',
 '107334',
 '107511',
 '10887',
 '108989',
 '109267',
 '109347',
 '10937',
 '10969',
 '110062',
 '110499',
 '110956',
 '112631',
 '113565',
 '11464',
 '115072',
 '115504',
 '116754',
 '11719',
 '118501',
 '120142',
 '12233',
 '122926',
 '123038',
 '12514',
 '127579',
 '127698',
 '128254',
 '129232',
 '129509',
 '131855',
 '132410',
 '133932',
 '13395',
 '13475',
 '1358',
 '136933',
 '137573',
 '138169',
 '13830',
 '140725',
 '140794',
 '14090',
 '14092',
 '140976',
 '141008',
 '14105',
 '142261',
 '143062',
 '14314',
 '143294',
 '143817',
 '144598',
 '1446',
 '14532',
 '145352',
 '145415',
 '145567',
 '145672',
 '145773',
 '146635',
 '146640',
 '147581',
 '147610',
 '147972',
 '147997',
 '148126',
 '14849',
 '149342',
 '149404',
 '149708',
 '14986',
 '150473',
 '151160',
 '151211',
 '151453',
 '151497',
 '151630',
 '151663',
 '152061',
 '152792',
 '152847',
 

On to the BM25 TF-IDF representation, we'll create the td-idf matrix for terms-documents, first without the query component. 

The query component is dependent on the terms in our query. So we'll just calculate that, and multiply it with the overall score when we want to retreive documents for a particular query.

In [14]:
#Building a TF-IDF representation using BM25 

NO_DOCS = len(documents) #Number of documents

AVG_LEN_DOC = sum([len(doc) for doc in documents.values()])/len(documents) #Average length of documents

#The function below takes the documentid, and the term, to calculate scores for the tf and idf
#components, and multiplies them together.
def tf_idf_score(k1,b,term,docid):  
    
    ft = len(inverted_index[term]) 
    term = stemmer.stem(term.lower())
    fdt =  documents[docid].count(term)
    
    idf_comp = math.log((NO_DOCS - ft + 0.5)/(ft+0.5))
    
    tf_comp = ((k1 + 1)*fdt)/(k1*((1-b) + b*(len(documents[docid])/AVG_LEN_DOC))+fdt)
    
    return idf_comp * tf_comp

#Method to create tf_idf matrix without the query component
def create_tf_idf(k1,b):
    tf_idf = defaultdict(dict)
    for term in set(inverted_index.keys()):
        for docid in inverted_index[term]:
            tf_idf[term][docid] = tf_idf_score(k1,b,term,docid)
    return tf_idf

In [15]:
#Creating tf_idf matrix with said parameter values: k1 and b for all documents.
tf_idf = create_tf_idf(1.5,0.5)

We took the default values for k1 and b (1.5 and 0.5), which seemed to give good results. Although these parameters may be altered depending on the type of data being dealth with. 

Now we create a method to retrieve the query component, and another method that will use the previous ones and retrieve the relevant documents for a query, sorted on the basis of their ranks. 

In [16]:
#Method to retrieve query component
def get_qtf_comp(k3,term,fqt):
    return ((k3+1)*fqt[term])/(k3 + fqt[term])


#Method to retrieve documents || Returns a set of documents and their relevance scores. 
def retr_docs(query,result_count):
    q_terms = [stemmer.stem(term.lower()) for term in query.split() if term not in stopwords] #Removing stopwords from queries
    fqt = {}
    for term in q_terms:
        fqt[term] = fqt.get(term,0) + 1
    
    scores = {}
    
    for word in fqt.keys():
        #print word + ': '+ str(inverted_index[word])
        for document in inverted_index[word]:
            scores[document] = scores.get(document,0) + (tf_idf[word][document]*get_qtf_comp(0,word,fqt)) #k3 chosen as 0 (default)
    
    return sorted(scores.items(),key = lambda x : x[1] , reverse=True)[:result_count]        

Let's try and retrieve a document for a query. 

In [18]:
retr_docs("Manchester United",5)

[('19961', 12.570721363284687),
 ('83266', 12.500367334396838),
 ('266959', 12.46418348068098),
 ('20206', 12.324327863972716),
 ('253314', 12.008548114449386)]

Checking out the terms in the top ranked document..

In [21]:
documents['19961']

[u'manchest',
 u'unit',
 u'manchest',
 u'unit',
 u'footbal',
 u'club',
 u'english',
 u'profession',
 u'footbal',
 u'club',
 u'base',
 u'old',
 u'trafford',
 u'greater',
 u'manchest',
 u'play',
 u'premier',
 u'leagu',
 u'found',
 u'newton',
 u'heath',
 u'lyr',
 u'footbal',
 u'club',
 u'club',
 u'chang',
 u'name',
 u'manchest',
 u'unit',
 u'move',
 u'old',
 u'trafford',
 u'manchest',
 u'unit',
 u'mani',
 u'trophi',
 u'english',
 u'footbal',
 u'includ',
 u'record',
 u'leagu',
 u'titl',
 u'record',
 u'fa',
 u'cup',
 u'four',
 u'leagu',
 u'cup',
 u'record',
 u'fa',
 u'commun',
 u'shield',
 u'club',
 u'also',
 u'three',
 u'european',
 u'cup',
 u'one',
 u'uefa',
 u'cup',
 u'winner',
 u'cup',
 u'one',
 u'uefa',
 u'super',
 u'cup',
 u'one',
 u'intercontinent',
 u'cup',
 u'one',
 u'fifa',
 u'club',
 u'world',
 u'cup',
 u'--',
 u'club',
 u'continent',
 u'trebl',
 u'premier',
 u'leagu',
 u'fa',
 u'cup',
 u'uefa',
 u'champion',
 u'leagu',
 u'unpreced',
 u'feat',
 u'english',
 u'club',
 u'munich',
 

We find out the the retrieval engine has worked quite well in this case. The top ranked document for the query is a snippet of the wikipedia article for Manchester United Football Club. 

On further inspection, we can see that the documents ranked lower are, for example, for The University of Manchester, or even just articles with the words 'Manchester' or 'United' in them.

Now we can begin translating the German queries to English.